## Models, Prompts & Parsers

### Model

In [ ]:
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage

def get_completion(prompt, model="gemma3:12b"):
    llm = ChatOllama(model=model, temperature=0)
    # Create a list of messages the simple way ChatOllama class expects
    messages = [HumanMessage(content=prompt)]
    # Call the model with the formatted messages
    response = llm(messages)
    return response.content

### Tests

In [ ]:
model = "gemma3:12b"
prompt = "What is 1+1?"
get_completion(prompt, model)

'1 + 1 = 2\n'

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)
get_completion(prompt, model)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



'Okay, here\'s a translation of the text into American English, maintaining a calm and respectful tone:\n\n"Oh dear, I\'m quite frustrated! The lid flew off my blender and splattered smoothie all over my kitchen walls. To top it off, the warranty doesn\'t cover the cost of cleaning up the mess. I could really use some assistance right now."\n\n**Key changes and why:**\n\n*   **"Arrr, I be fuming"** became "Oh dear, I\'m quite frustrated" - This replaces the pirate-like expression with a more standard and less dramatic phrasing.\n*   **"me"** became "my" - Corrected the possessive pronoun.\n*   **"don\'t cover the cost of cleaning up me kitchen"** became "doesn\'t cover the cost of cleaning up the mess" - More natural phrasing and avoids the informal "me."\n*   **"matey"** became "right now" - Replaced the informal term with a more direct request for help.\n*   The overall tone is softened to be more polite and less agitated.'

### Templating

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [ ]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [ ]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


### Chatting

In [ ]:
chat = ChatOllama(model=model, temperature=0)
chat

ChatOllama(model='gemma3:12b', temperature=0.0)

In [ ]:
customer_response = chat(customer_messages)
print(customer_response.content)

Okay, here's a translation of the text into American English, maintaining a calm and respectful tone:

"Oh dear, I'm quite frustrated! The lid flew off my blender and splattered smoothie all over my kitchen walls. To top it off, the warranty doesn't cover the cost of cleaning up the mess. I could really use some assistance right now."

**Key changes and why:**

*   **"Arrr, I be fuming"** became "Oh dear, I'm quite frustrated" - This replaces the pirate-like exclamation with a more standard and less dramatic expression of annoyance.
*   **"me"** became "my" - Corrected the possessive pronoun.
*   **"don't cover the cost of cleaning up me kitchen"** became "doesn't cover the cost of cleaning up the mess" - More natural phrasing and avoids the informal "me."
*   **"matey"** became "right now" - Removed the pirate term and replaced it with a more direct request for help.
*   The overall tone is softened to be more polite and less demanding.


In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [ ]:
service_response = chat(service_messages)
print(service_response.content)

Avast there, good sir/madam!

We be regretfully informin' ye that yer kitchen cleanin' costs be not covered under the warranty. 'Tis due to a mishap o' yer own makin', forgettin' to secure the lid afore operatin' yer blender. A sorry state o' affairs, aye, but alas, the warranty does not extend to such occurrences. 

Fair winds and safe sailin', and may yer future blendin' be more careful! Huzzah!


### Formatting output parsing

In [ ]:
#  let's say we want  an llm to output a json like this:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# ask the llm to parse the review & extract the information as well as output it in json format
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
# we pass the template to the `from_template` method
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [ ]:
# and pass the customer review to the template
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOllama(model=model, temperature=0)
response = chat(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there",
    "but I think it's worth it for the extra features"
  ]
}
```


In [ ]:
# looks like a json but is in fact a string
print(type(response.content))
# so will raise an error if we try to access it as a dictionary
response.content.get('gift')

<class 'str'>


AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
# we need to specify the output schema
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
# the actual constructed prompt
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "['It's slightly more expensive than the other leaf blowers out there', 'but I think it's worth it for the extra features']"
}
```


In [ ]:
# we can use the output parser to parse the response
output_dict = output_parser.parse(response.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "['It's slightly more expensive than the other leaf blowers out there', 'but I think it's worth it for the extra features']"}

In [ ]:
# which is of type dict
print(type(output_dict))

<class 'dict'>


In [ ]:
# that can be accessed like a proper dictionary
output_dict.get('delivery_days')

'2'